Note: All the cells in this notebook are self-contained, so they can be run independently. Before starting, make sure to remove the "_cache" directory, which we will use here, e.g.:

In [1]:
!rm -rf _cache

# Writing a plug-and-play caching decorator from scratch

Our goal is quite simple: we have some slow function `f(a: int, b: int)`. Say we call `f(10, 42)` today. It will be slow. We will be annoyed. Now say in three months we call `f(10, 42)` again. It would be bad to have to wait for `f(10, 42)` to run again, since we already ran it three months ago. But unless we do anything about it, we will have to wait. Here's how this looks like:

In [2]:
import time

def f(a: int, b: int):
    time.sleep(2)
    result = a + b
    return result

# Today
print(f(10, 42))  # slow

# In three months
print(f(10, 42))  # slow again!

52
52


# Level 1: What we do in all our CS classes

The solution to our problem is ubiquitous and is featured in almost any CS class we encounter: caching. In its simplest form (which is the main tool in dynamic programming), we can just create a dictionary with previously computed values and use it, as follows:

In [3]:
import time

f_cache = {}

def f(a: int, b: int):
    if (a, b) in f_cache:
        return f_cache[(a, b)]
    else:
        time.sleep(2)
        result = a + b
        f_cache[(a, b)] = result
        return result

# Today
print(f(10, 42))  # slow

# In three months
print(f(10, 42))  # fast, yay!

52
52


# Level 2: Persistent cache by using the filesystem

So far so good, but we have a huge issue: The cache lives in memory, so if we restart our computer, we loose all the cached data! To address this, we can use the filesystem as a persistent cache, storing the value of `f(a, b)` in the location `_cache/f/a/{a}/b/{b}/result.txt`, as follows:

In [4]:
import time
import os

def f(a: int, b: int):
    # Determine the path where the cached value should be
    cached_result_path = f"_cache/f/a/{a}/b/{b}/result.txt"
    # If the cached value is present, return it.
    if os.path.exists(cached_result_path):
        return int(open(cached_result_path, "r").read())
    else:
        # Only if the cached value is not present do we do any computation.
        time.sleep(2)
        result = a + b
        os.makedirs(f"_cache/f/a/{a}/b/{b}")
        open(cached_result_path, "w").write(str(result))
        return result

# Today
print(f(10, 42))  # slow

# In three months
print(f(10, 42))  # fast, yay!

52
52


# Level 3 (most important!): Using decorators to streamline code

This is great, but it has a serious design limitation: Our function used to have 3 lines of code, and now it has 9. Most of the code just has to do with caching, obscuring what the function is really doing. Furthermore, if we had two functions `f(a, b)` and `g(a, b, c)` that we wanted to cache, we would need to modify both in the same way, creating significant code duplication and obfuscation, such as:

In [5]:
import time
import os

def f(a: int, b: int):
    # Determine the path where the cached value should be
    cached_result_path = f"_cache/f/a/{a}/b/{b}/result.txt"
    # If the cached value is present, return it.
    if os.path.exists(cached_result_path):
        return int(open(cached_result_path, "r").read())
    else:
        # Only if the cached value is not present do we do any computation.
        time.sleep(2)
        result = a + b
        os.makedirs(f"_cache/f/a/{a}/b/{b}")
        open(cached_result_path, "w").write(str(result))
        return result

def g(a: int, b: int, c: int):
    # Determine the path where the cached value should be
    cached_result_path = f"_cache/g/a/{a}/b/{b}/c/{c}/result.txt"
    # If the cached value is present, return it.
    if os.path.exists(cached_result_path):
        return int(open(cached_result_path, "r").read())
    else:
        # Only if the cached value is not present do we do any computation.
        time.sleep(2)
        result = a * b * c
        os.makedirs(f"_cache/g/a/{a}/b/{b}/c/{c}")
        open(cached_result_path, "w").write(str(result))
        return result

# Today
print(f(100, 42))  # slow
print(g(100, 42, 2))  # slow

# In three months
print(f(100, 42))  # fast, yay!
print(g(100, 42, 2))  # fast, yay!

142
8400
142
8400


We will see how to avoid code duplication by writing a function `caching_decorator` that given any **arbitrary** function `func`, returns the version of `func` modified to do caching as above. So, `caching_decorator(f)` would return the equivalent of:

```
def f(a: int, b: int):
    # Determine the path where the cached value should be
    cached_result_path = f"_cache/f/a/{a}/b/{b}/result.txt"
    # If the cached value is present, return it.
    if os.path.exists(cached_result_path):
        return int(open(cached_result_path, "r").read())
    else:
        # Only if the cached value is not present do we do any computation.
        time.sleep(2)
        result = a + b
        os.makedirs(f"_cache/f/a/{a}/b/{b}")
        open(cached_result_path, "w").write(str(result))
        return result
```

while `caching_decorator(g)` would return the equivalent of:

```
def g(a: int, b: int, c: int):
    # Determine the path where the cached value should be
    cached_result_path = f"_cache/g/a/{a}/b/{b}/c/{c}/result.txt"
    # If the cached value is present, return it.
    if os.path.exists(cached_result_path):
        return int(open(cached_result_path, "r").read())
    else:
        # Only if the cached value is not present do we do any computation.
        time.sleep(2)
        result = a * b * c
        os.makedirs(f"_cache/g/a/{a}/b/{b}/c/{c}")
        open(cached_result_path, "w").write(str(result))
        return result
```

How do we do this? The trick is to use python's `inspect` module, which provides an easy way to get the argument names and values of a function call. For example:

In [6]:
from typing import Any, List, Tuple
from inspect import signature

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of when calling `func` with `args` and `kwargs`.
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

# Example usage:
def h(d, c, b, a = 1):
    return d + c + b + a

args = (4, 3)
kwargs = {'b': 2}

print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]


[('d', 4), ('c', 3), ('b', 2), ('a', 1)]


With the function `get_argnames_and_values`, now we can write a very generic caching decorator. To allow for arbitrary outputs, we will use `pickle` to cache the results:

In [7]:
import time
import os
import pickle

from typing import Any, List, Tuple
from inspect import signature

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def caching_decorator(func):
    """
    Create a version of `func` with caching.
    """
    def func_with_caching(*args, **kwargs):
        # Determine the path where the cached value should be
        argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
        cached_result_path = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values) + "/result.txt"
        # If the cached value is present, return it.
        if os.path.exists(cached_result_path):
            with open(cached_result_path, "rb") as f:
                return pickle.load(f)
        else:
            # Only if the cached value is not present do we do any computation.
            result = func(*args, **kwargs)
            os.makedirs(f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values))
            with open(cached_result_path, "wb") as f:
                pickle.dump(result, f)
                f.flush()
            return result
    return func_with_caching

def f(a: int, b: int):
    time.sleep(2)
    result = a + b
    return result
f = caching_decorator(f)

def g(a: int, b: int, c: int):
    time.sleep(2)
    result = a * b * c
    return result
g = caching_decorator(g)

# Today
print(f(1000, 42))  # slow
print(g(1000, 42, 2))  # slow

# In three months
print(f(1000, 42))  # fast, yay!
print(g(1000, 42, 2))  # fast, yay!

1042
84000
1042
84000


Awesome! Now we can add caching to any function with just one line of code! Before we continue though, an important note. The pattern that we just used, namely:
```
def f(...):
    ...
f = decorator(f)
```
is so common in python, that python has reserved syntax for this. The syntax is the following:
```
@decorator
def f(...):
    ...
```

Therefore, we can rewrite our code as follows:

In [8]:
import time
import os
import pickle

from typing import Any, List, Tuple
from inspect import signature

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def caching_decorator(func):
    """
    Create a version of `func` with caching.
    """
    def func_with_caching(*args, **kwargs):
        # Determine the path where the cached value should be
        argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
        cached_result_path = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values) + "/result.txt"
        # If the cached value is present, return it.
        if os.path.exists(cached_result_path):
            with open(cached_result_path, "rb") as f:
                return pickle.load(f)
        else:
            # Only if the cached value is not present do we do any computation.
            result = func(*args, **kwargs)
            os.makedirs(f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values))
            with open(cached_result_path, "wb") as f:
                pickle.dump(result, f)
                f.flush()
            return result
    return func_with_caching

@caching_decorator
def f(a: int, b: int):
    time.sleep(2)
    result = a + b
    return result

@caching_decorator
def g(a: int, b: int, c: int):
    time.sleep(2)
    result = a * b * c
    return result

# Today
print(f(10000, 42))  # slow
print(g(10000, 42, 2))  # slow

# In three months
print(f(10000, 42))  # fast, yay!
print(g(10000, 42, 2))  # fast, yay!

10042
840000
10042
840000


This covers the core functionality of caching. Now, all that remains is to tweak it by adding more and more cool features.

# Level 4+: Tweaking the decorator with endless amounts of cool features

## Enforcing the use of keyword arguments

The first thing we would like to do is the following: it is generally good practice to always call functions using ONLY keyword arguments. Therefore, lets enforce it! To do this, we can just check if len(args) > 0, and if so raise an error:

In [9]:
import time
import os
import pickle

from typing import Any, List, Tuple
from inspect import signature

class CacheUsageError(Exception):
    pass

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def caching_decorator(func):
    """
    Create a version of `func` with caching.
    """
    def func_with_caching(*args, **kwargs):
        # Force the user to use keyword arguments only since it is good practice
        if len(args) > 0:
            raise CacheUsageError(
                f"Please call {func.__name__} with keyword arguments only. "
                f"Positional arguments are not allowed since it is bad practice."
            )
        # Determine the path where the cached value should be
        argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
        cached_result_path = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values) + "/result.txt"
        # If the cached value is present, return it.
        if os.path.exists(cached_result_path):
            with open(cached_result_path, "rb") as f:
                return pickle.load(f)
        else:
            # Only if the cached value is not present do we do any computation.
            result = func(*args, **kwargs)
            os.makedirs(f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values))
            with open(cached_result_path, "wb") as f:
                pickle.dump(result, f)
                f.flush()
            return result
    return func_with_caching

@caching_decorator
def f(a: int, b: int):
    time.sleep(2)
    result = a + b
    return result

@caching_decorator
def g(a: int, b: int, c: int):
    time.sleep(2)
    result = a * b * c
    return result

# Today
print(f(a=100000, b=42))  # slow
print(g(a=100000, b=42, c=2))  # slow

# In three months
print(f(a=100000, b=42))  # fast, yay!
print(g(a=100000, b=42, c=2))  # fast, yay!

print(g(100000, 42, 2))  # Error!

100042
8400000
100042
8400000


CacheUsageError: Please call g with keyword arguments only. Positional arguments are not allowed since it is bad practice.

## Exclusion of arguments

The next goodie we would like to have, is the possibility of **excluding some arguments from the cache path**. For example, things like `verbose` which are completely irrelevant and do not affect the result of the computation. Currently, these arguments are used to build the cache path. Thus, for example, we have this issue:

In [10]:
import time
import os
import pickle

from typing import Any, List, Tuple
from inspect import signature

class CacheUsageError(Exception):
    pass

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def caching_decorator(func):
    """
    Create a version of `func` with caching.
    """
    def func_with_caching(*args, **kwargs):
        # Force the user to use keyword arguments only since it is good practice
        if len(args) > 0:
            raise CacheUsageError(
                f"Please call {func.__name__} with keyword arguments only. "
                f"Positional arguments are not allowed since it is bad practice."
            )
        # Determine the path where the cached value should be
        argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
        cached_result_path = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values) + "/result.txt"
        # If the cached value is present, return it.
        if os.path.exists(cached_result_path):
            with open(cached_result_path, "rb") as f:
                return pickle.load(f)
        else:
            # Only if the cached value is not present do we do any computation.
            result = func(*args, **kwargs)
            os.makedirs(f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values))
            with open(cached_result_path, "wb") as f:
                pickle.dump(result, f)
                f.flush()
            return result
    return func_with_caching

@caching_decorator
def f(a: int, b: int, verbose: bool):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b
    return result

# Today
print(f(a=1000000, b=42, verbose=True))  # slow
print(f(a=1000000, b=42, verbose=False))  # Also slow!!!

Hi!
1000042
1000042


To fix this, we will allow for the user to prove a list of arguments to exclude from the cache path. We would like to be able to do this:

```
@caching_decorator(exclude_args=["verbose"])
def f(a: int, b: int, verbose: bool):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b
    return result
```

We can implement this feature with minor modifications as follows:

In [11]:
import time
import os
import pickle

from typing import Any, List, Tuple
from inspect import signature

class CacheUsageError(Exception):
    pass

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def caching_decorator(exclude_args: List[str] = []):
    """
    Create a caching decorator which excludes the `exclude_args`.
    """
    def caching_decorator_with_excluded_args(func):
        """
        Create a version of `func` with caching which excludes the `exclude_args`.
        """
        def func_with_caching(*args, **kwargs):
            # Force the user to use keyword arguments only since it is good practice
            if len(args) > 0:
                raise CacheUsageError(
                    f"Please call {func.__name__} with keyword arguments only. "
                    f"Positional arguments are not allowed since it is bad practice."
                )
            # Determine the path where the cached value should be
            argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
            cached_result_dir = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values if argname not in exclude_args)
            cached_result_path = cached_result_dir + "/result.txt"
            # If the cached value is present, return it.
            if os.path.exists(cached_result_path):
                with open(cached_result_path, "rb") as f:
                    return pickle.load(f)
            else:
                # Only if the cached value is not present do we do any computation.
                result = func(*args, **kwargs)
                os.makedirs(cached_result_dir)
                with open(cached_result_path, "wb") as f:
                    pickle.dump(result, f)
                    f.flush()
                return result
        return func_with_caching
    return caching_decorator_with_excluded_args

@caching_decorator(exclude_args=["verbose"])
def f(a: int, b: int, verbose: bool):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b
    return result

# Today
print(f(a=10000000, b=42, verbose=True))  # slow
print(f(a=10000000, b=42, verbose=False))  # Fast, yay!

Hi!
10000042
10000042


## Exclusion of arguments only when they are defaults

Let's continue to add more features. A common thing that might happen is that we initially write a function and many months later we extend it with a new argument in a way that is backwards compatible. For example:

In [12]:
import time
import os
import pickle

from typing import Any, List, Tuple
from inspect import signature

class CacheUsageError(Exception):
    pass

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def caching_decorator(exclude_args: List[str] = []):
    """
    Create a caching decorator which excludes the `exclude_args`.
    """
    def caching_decorator_with_excluded_args(func):
        """
        Create a version of `func` with caching which excludes the `exclude_args`.
        """
        def func_with_caching(*args, **kwargs):
            # Force the user to use keyword arguments only since it is good practice
            if len(args) > 0:
                raise CacheUsageError(
                    f"Please call {func.__name__} with keyword arguments only. "
                    f"Positional arguments are not allowed since it is bad practice."
                )
            # Determine the path where the cached value should be
            argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
            cached_result_dir = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values if argname not in exclude_args)
            cached_result_path = cached_result_dir + "/result.txt"
            # If the cached value is present, return it.
            if os.path.exists(cached_result_path):
                with open(cached_result_path, "rb") as f:
                    return pickle.load(f)
            else:
                # Only if the cached value is not present do we do any computation.
                result = func(*args, **kwargs)
                os.makedirs(cached_result_dir)
                with open(cached_result_path, "wb") as f:
                    pickle.dump(result, f)
                    f.flush()
                return result
        return func_with_caching
    return caching_decorator_with_excluded_args

@caching_decorator(exclude_args=["verbose"])
def f(a: int, b: int, verbose: bool):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b
    return result

# Today
print(f(a=100000000, b=42, verbose=True))  # slow
print(f(a=100000000, b=42, verbose=False))  # Fast, yay!

# In three months, we extend f in a backwards compatible way
@caching_decorator(exclude_args=["verbose"])
def f(a: int, b: int, verbose: bool, c: int = 0):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b + c
    return result

print(f(a=100000000, b=42, verbose=False))  # Slow again!!!

Hi!
100000042
100000042
100000042


The issue is that by adding a new argument -- event if it has a backwards compatible default -- it will be used to determine the cache path. Therefore, we need to support the exclusion of arguments if they have a default value. We would like to do something like this:

```
# In three months, we extend f in a backwards compatible way
@caching_decorator(exclude_args=["verbose"], exclude_args_if_default = ["c"])
def f(a: int, b: int, verbose: bool, c: int = 0):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b + c
    return result
```

We can achieve this easily with minor modifications as follows:

In [13]:
import time
import os
import pickle

from typing import Any, List, Tuple
from inspect import signature

class CacheUsageError(Exception):
    pass

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def get_filtered_argnames_and_values(func, exclude_args, exclude_args_if_default, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call, applying the necessary exclusions.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2, 'a': 1}

    print(get_filtered_argnames_and_values(h, ["c"], ["a"], *args, **kwargs))  # => prints [('d', 4), ('b', 2)]
    """
    argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
    def should_exclude_arg(argname: str, value: Any) -> bool:
        """
        Determine if we should exclude `argname` given that its value is `value`.
        """
        # First handle exclude_args
        if argname in exclude_args:
            return True
        # Now handle exclude_args_if_default
        elif argname in exclude_args_if_default:
            s = signature(func)
            p = s.parameters[argname]
            default = p.default
            if value == default:
                return True
        else:
            return False
    filtered_argnames_and_values = [(argname, value) for (argname, value) in argnames_and_values if not should_exclude_arg(argname, value)]
    return filtered_argnames_and_values

def caching_decorator(exclude_args: List[str] = [], exclude_args_if_default: List[str] = []):
    """
    Create a caching decorator which excludes the `exclude_args`,
    and only excludes `exclude_args_if_default` if they take on their default value.
    """
    def caching_decorator_with_excluded_args(func):
        """
        Create a version of `func` with caching which excludes the `exclude_args`,
        and only excludes `exclude_args_if_default` if they take on their default value.
        """
        def func_with_caching(*args, **kwargs):
            # Force the user to use keyword arguments only since it is good practice
            if len(args) > 0:
                raise CacheUsageError(
                    f"Please call {func.__name__} with keyword arguments only. "
                    f"Positional arguments are not allowed since it is bad practice."
                )
            # Determine the path where the cached value should be
            argnames_and_values = get_filtered_argnames_and_values(func, exclude_args, exclude_args_if_default, *args, **kwargs)
            cached_result_dir = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values)
            cached_result_path = cached_result_dir + "/result.txt"
            # If the cached value is present, return it.
            if os.path.exists(cached_result_path):
                with open(cached_result_path, "rb") as f:
                    return pickle.load(f)
            else:
                # Only if the cached value is not present do we do any computation.
                result = func(*args, **kwargs)
                os.makedirs(cached_result_dir)
                with open(cached_result_path, "wb") as f:
                    pickle.dump(result, f)
                    f.flush()
                return result
        return func_with_caching
    return caching_decorator_with_excluded_args

@caching_decorator(exclude_args=["verbose"])
def f(a: int, b: int, verbose: bool):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b
    return result

# Today
print(f(a=1000000000, b=42, verbose=True))  # slow
print(f(a=1000000000, b=42, verbose=False))  # Fast, yay!

# In three months, we extend f in a backwards compatible way
@caching_decorator(exclude_args=["verbose"], exclude_args_if_default=["c"])
def f(a: int, b: int, verbose: bool, c: int = 0):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b + c
    return result

print(f(a=1000000000, b=42, verbose=False))  # Also fast, yay!

Hi!
1000000042
1000000042
1000000042


## Avoiding data corruption

Besides `result.txt`, we can write other files in this directory for all sorts of different purposes. A great example is the use of "success tokens" to make caching operations atomic and avoid data corruption. Imagine that `result.txt` is being written but suddenly the power goes out. The file `result.txt` will exist but it will be corrupted (partially written)! When we try to read the file, we will get an unpickling error. To avoid this, we can use success tokens. The idea is simple: first we write `result.txt`. Then, we create the file `success.txt`. When reading from the cache, to determine whether the data is corrupted or not, we need to check whether `success.txt` exists. If it doesn't, regardless of whether `result.txt` exists, we need to redo the computation. If `success.txt` exists, we know that we are good, and we can unpickle and return the contents of `result.txt`. We do this below by adding small amounts of logic:

In [14]:
import time
import os
import pickle

from typing import Any, List, Tuple
from inspect import signature

class CacheUsageError(Exception):
    pass

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def get_filtered_argnames_and_values(func, exclude_args, exclude_args_if_default, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call, applying the necessary exclusions.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2, 'a': 1}

    print(get_filtered_argnames_and_values(h, ["c"], ["a"], *args, **kwargs))  # => prints [('d', 4), ('b', 2)]
    """
    argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
    def should_exclude_arg(argname: str, value: Any) -> bool:
        """
        Determine if we should exclude `argname` given that its value is `value`.
        """
        # First handle exclude_args
        if argname in exclude_args:
            return True
        # Now handle exclude_args_if_default
        elif argname in exclude_args_if_default:
            s = signature(func)
            p = s.parameters[argname]
            default = p.default
            if value == default:
                return True
        else:
            return False
    filtered_argnames_and_values = [(argname, value) for (argname, value) in argnames_and_values if not should_exclude_arg(argname, value)]
    return filtered_argnames_and_values

def caching_decorator(exclude_args: List[str] = [], exclude_args_if_default: List[str] = []):
    """
    Create a caching decorator which excludes the `exclude_args`,
    and only excludes `exclude_args_if_default` if they take on their default value.
    """
    def caching_decorator_with_excluded_args(func):
        """
        Create a version of `func` with caching which excludes the `exclude_args`,
        and only excludes `exclude_args_if_default` if they take on their default value.
        """
        def func_with_caching(*args, **kwargs):
            # Force the user to use keyword arguments only since it is good practice
            if len(args) > 0:
                raise CacheUsageError(
                    f"Please call {func.__name__} with keyword arguments only. "
                    f"Positional arguments are not allowed since it is bad practice."
                )
            # Determine the path where the cached value should be
            argnames_and_values = get_filtered_argnames_and_values(func, exclude_args, exclude_args_if_default, *args, **kwargs)
            cached_result_dir = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values)
            cached_result_path = cached_result_dir + "/result.txt"
            success_token_path = cached_result_dir + "/success.txt"
            # If the cached value is present AND THE SUCCESS TOKEN EXISTS, return it.
            if os.path.exists(cached_result_path) and os.path.exists(success_token_path):
                with open(cached_result_path, "rb") as f:
                    return pickle.load(f)
            else:
                # If the result.txt file is present but the success token is missing, log a warning
                if os.path.exists(cached_result_path) and not os.path.exists(success_token_path):
                    print(f"Possibly corrupted cache file: '{cached_result_path}'. Going to re-run.")
                # Only if the cached value is not present do we do any computation.
                result = func(*args, **kwargs)
                os.makedirs(cached_result_dir, exist_ok=True)
                with open(cached_result_path, "wb") as f:
                    pickle.dump(result, f)
                    f.flush()
                with open(success_token_path, "w") as f:
                    f.write("SUCCESS\n")  # Just to put some content into the file.
                    f.flush()
                return result
        return func_with_caching
    return caching_decorator_with_excluded_args


@caching_decorator(exclude_args=["verbose"], exclude_args_if_default=["c"])
def f(a: int, b: int, verbose: bool, c: int = 0):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b + c
    return result

print(f(a=10000000000, b=42, verbose=False))  # Slow
# Let's simulate a data corruption scenario by removing the success token file
os.system("rm _cache/f/a/10000000000/b/42/success.txt")
print(f(a=10000000000, b=42, verbose=False))  # Will need to re-run bc of possible data corruption! Slow again.
print(f(a=10000000000, b=42, verbose=False))  # Now fast, yay!

10000000042
Possibly corrupted cache file: '_cache/f/a/10000000000/b/42/result.txt'. Going to re-run.
10000000042
10000000042


## Using a logger

Using print statements is not a great idea. We should use the python `logging` module instead. We make this change below:

In [15]:
import time
import os
import pickle
import logging
import sys

from typing import Any, List, Tuple
from inspect import signature


def _init_logger():
    logger = logging.getLogger('.'.join(__name__.split('.')[:-1]))
    logger.setLevel(logging.INFO)
    fmt_str = "[%(asctime)s] - %(name)s - %(levelname)s - %(message)s"
    formatter = logging.Formatter(fmt_str)

    consoleHandler = logging.StreamHandler(sys.stdout)
    consoleHandler.setFormatter(formatter)
    logger.addHandler(consoleHandler)


_init_logger()
logger = logging.getLogger('.'.join(__name__.split('.')[:-1]))


class CacheUsageError(Exception):
    pass

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def get_filtered_argnames_and_values(func, exclude_args, exclude_args_if_default, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call, applying the necessary exclusions.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2, 'a': 1}

    print(get_filtered_argnames_and_values(h, ["c"], ["a"], *args, **kwargs))  # => prints [('d', 4), ('b', 2)]
    """
    argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
    def should_exclude_arg(argname: str, value: Any) -> bool:
        """
        Determine if we should exclude `argname` given that its value is `value`.
        """
        # First handle exclude_args
        if argname in exclude_args:
            return True
        # Now handle exclude_args_if_default
        elif argname in exclude_args_if_default:
            s = signature(func)
            p = s.parameters[argname]
            default = p.default
            if value == default:
                return True
        else:
            return False
    filtered_argnames_and_values = [(argname, value) for (argname, value) in argnames_and_values if not should_exclude_arg(argname, value)]
    return filtered_argnames_and_values

def caching_decorator(exclude_args: List[str] = [], exclude_args_if_default: List[str] = []):
    """
    Create a caching decorator which excludes the `exclude_args`,
    and only excludes `exclude_args_if_default` if they take on their default value.
    """
    def caching_decorator_with_excluded_args(func):
        """
        Create a version of `func` with caching which excludes the `exclude_args`,
        and only excludes `exclude_args_if_default` if they take on their default value.
        """
        def func_with_caching(*args, **kwargs):
            # Force the user to use keyword arguments only since it is good practice
            if len(args) > 0:
                raise CacheUsageError(
                    f"Please call {func.__name__} with keyword arguments only. "
                    f"Positional arguments are not allowed since it is bad practice."
                )
            # Determine the path where the cached value should be
            argnames_and_values = get_filtered_argnames_and_values(func, exclude_args, exclude_args_if_default, *args, **kwargs)
            cached_result_dir = f"_cache/{func.__name__}/" + "/".join(f"{argname}/{val}" for argname, val in argnames_and_values)
            cached_result_path = cached_result_dir + "/result.txt"
            success_token_path = cached_result_dir + "/success.txt"
            # If the result.txt file is present AND THE SUCCESS TOKEN EXISTS, return it.
            if os.path.exists(cached_result_path) and os.path.exists(success_token_path):
                with open(cached_result_path, "rb") as f:
                    return pickle.load(f)
            else:
                # If the result.txt file is present but the success token is missing, log a warning
                if os.path.exists(cached_result_path) and not os.path.exists(success_token_path):
                    logger.info(f"Possibly corrupted cache file: '{cached_result_path}'. Going to re-run.")
                
                # TODO
                # If the success token exists but there is not result.txt file, then nothing makes sense! Throw an error.
                if not os.path.exists(cached_result_path) and os.path.exists(success_token_path):
                    logger.info(f"Possibly corrupted cache file: '{cached_result_path}'. Going to re-run.")
                
                # Only if the cached value is not present do we do any computation.
                result = func(*args, **kwargs)
                os.makedirs(cached_result_dir, exist_ok=True)
                with open(cached_result_path, "wb") as f:
                    pickle.dump(result, f)
                    f.flush()
                with open(success_token_path, "w") as f:
                    f.write("SUCCESS\n")  # Just to put some content into the file.
                    f.flush()
                return result
        return func_with_caching
    return caching_decorator_with_excluded_args


@caching_decorator(exclude_args=["verbose"], exclude_args_if_default=["c"])
def f(a: int, b: int, verbose: bool, c: int = 0):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = a + b + c
    return result

print(f(a=10000000000, b=42, verbose=False))  # Slow
# Let's simulate a data corruption scenario by removing the success token file
os.system("rm _cache/f/a/10000000000/b/42/success.txt")
print(f(a=10000000000, b=42, verbose=False))  # Will need to re-run bc of possible data corruption! Slow again.
print(f(a=10000000000, b=42, verbose=False))  # Now fast, yay!

10000000042
[2023-11-09 09:52:57,643] - root - INFO - Possibly corrupted cache file: '_cache/f/a/10000000000/b/42/result.txt'. Going to re-run.
10000000042
10000000042


## Using hashes to build the cache path

Things look great so far, but we have a big issue: what if our function takes in a long list of numbers? For example a list of 1000 integers? Operating systems typically have a limit to the length of file paths. To be able to cache almsot everything, a key idea is to create the output path not by naively concatenating all the arguments, but by **hashing** all the arguments. For example:

In [16]:
import hashlib

def hash_all(xs: List[Any]) -> str:
    """
    Hash a list of values.
    """
    hashes = [hashlib.sha512(str(x).encode("utf-8")).hexdigest() for x in xs]
    res = hashlib.sha512("".join(hashes).encode("utf-8")).hexdigest()
    return res

print(
    hash_all(
        [
            "a", 10000000000, "b", 42, "c", "2", "d", [1, 10, 100, 123],
        ]
    )
)

66ea9c32fd839cb74a6a5f0ac5978af2f685e286ce1d48dcd2f9161239ce612bba644be150001e75cacd7e16b0b50922156b2f89987708d6e71f7dd89acf2254


By using this function, we can create a hash of the arguments to determine the output path. This way, by making a small modification to the `cached_result_dir` we obtain:

In [17]:
import time
import os
import pickle
import logging
import sys
import hashlib

from typing import Any, List, Tuple
from inspect import signature


def _init_logger():
    logger = logging.getLogger('.'.join(__name__.split('.')[:-1]))
    logger.setLevel(logging.INFO)
    fmt_str = "[%(asctime)s] - %(name)s - %(levelname)s - %(message)s"
    formatter = logging.Formatter(fmt_str)

    consoleHandler = logging.StreamHandler(sys.stdout)
    consoleHandler.setFormatter(formatter)
    logger.addHandler(consoleHandler)


_init_logger()
logger = logging.getLogger('.'.join(__name__.split('.')[:-1]))


class CacheUsageError(Exception):
    pass

def get_argnames_and_values(func, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2}

    print(get_argnames_and_values(h, *args, **kwargs))  # => prints [('d', 4), ('c', 3), ('b', 2), ('a', 1)]
    """
    binding = signature(func).bind(*args, **kwargs)
    binding.apply_defaults()
    argnames_and_values = list(binding.arguments.items())
    return argnames_and_values

def get_filtered_argnames_and_values(func, exclude_args, exclude_args_if_default, *args, **kwargs) -> List[Tuple[str, Any]]:
    """
    Get the argnames and values of a function call, applying the necessary exclusions.

    # Example usage:
    def h(d, c, b, a = 1):
        return d + c + b + a

    args = (4, 3)
    kwargs = {'b': 2, 'a': 1}

    print(get_filtered_argnames_and_values(h, ["c"], ["a"], *args, **kwargs))  # => prints [('d', 4), ('b', 2)]
    """
    argnames_and_values = get_argnames_and_values(func, *args, **kwargs)
    def should_exclude_arg(argname: str, value: Any) -> bool:
        """
        Determine if we should exclude `argname` given that its value is `value`.
        """
        # First handle exclude_args
        if argname in exclude_args:
            return True
        # Now handle exclude_args_if_default
        elif argname in exclude_args_if_default:
            s = signature(func)
            p = s.parameters[argname]
            default = p.default
            if value == default:
                return True
        else:
            return False
    filtered_argnames_and_values = [(argname, value) for (argname, value) in argnames_and_values if not should_exclude_arg(argname, value)]
    return filtered_argnames_and_values

def hash_all(xs: List[Any]) -> str:
    """
    Hash a list of values.
    """
    hashes = [hashlib.sha512(str(x).encode("utf-8")).hexdigest() for x in xs]
    res = hashlib.sha512("".join(hashes).encode("utf-8")).hexdigest()
    return res

def caching_decorator(exclude_args: List[str] = [], exclude_args_if_default: List[str] = []):
    """
    Create a caching decorator which excludes the `exclude_args`,
    and only excludes `exclude_args_if_default` if they take on their default value.
    """
    def caching_decorator_with_excluded_args(func):
        """
        Create a version of `func` with caching which excludes the `exclude_args`,
        and only excludes `exclude_args_if_default` if they take on their default value.
        """
        def func_with_caching(*args, **kwargs):
            # Force the user to use keyword arguments only since it is good practice
            if len(args) > 0:
                raise CacheUsageError(
                    f"Please call {func.__name__} with keyword arguments only. "
                    f"Positional arguments are not allowed since it is bad practice."
                )
            # Determine the path where the cached value should be
            argnames_and_values = get_filtered_argnames_and_values(func, exclude_args, exclude_args_if_default, *args, **kwargs)
            cached_result_dir = f"_cache/{func.__name__}/" + hash_all(sum([[argname, val] for (argname, val) in argnames_and_values], []))
            cached_result_path = cached_result_dir + "/result.txt"
            success_token_path = cached_result_dir + "/success.txt"
            # If the result.txt file is present AND THE SUCCESS TOKEN EXISTS, return it.
            if os.path.exists(cached_result_path) and os.path.exists(success_token_path):
                with open(cached_result_path, "rb") as f:
                    return pickle.load(f)
            else:
                # If the result.txt file is present but the success token is missing, log a warning
                if os.path.exists(cached_result_path) and not os.path.exists(success_token_path):
                    logger.info(f"Possibly corrupted cache file: '{cached_result_path}'. Going to re-run.")
                
                # TODO
                # If the success token exists but there is not result.txt file, then nothing makes sense! Throw an error.
                if not os.path.exists(cached_result_path) and os.path.exists(success_token_path):
                    logger.info(f"Possibly corrupted cache file: '{cached_result_path}'. Going to re-run.")
                
                # Only if the cached value is not present do we do any computation.
                result = func(*args, **kwargs)
                os.makedirs(cached_result_dir, exist_ok=True)
                with open(cached_result_path, "wb") as f:
                    pickle.dump(result, f)
                    f.flush()
                with open(success_token_path, "w") as f:
                    f.write("SUCCESS\n")  # Just to put some content into the file.
                    f.flush()
                return result
        return func_with_caching
    return caching_decorator_with_excluded_args


@caching_decorator(exclude_args=["verbose"])
def f(numbers_to_add: List[int], verbose: bool = False):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = sum(numbers_to_add)
    return result

@caching_decorator(exclude_args=["verbose"])
def g(numbers_to_multiply: List[int], verbose: bool = False):
    if verbose:
        print("Hi!")
    time.sleep(2)
    result = 1
    for x in numbers_to_multiply:
        result *= x
    return result

# Today
print(f(numbers_to_add=[1, 10, 100, 1234]))  # slow
print(g(numbers_to_multiply=[1, 2, 3, 4, 5]))  # slow

# In three months
print(f(numbers_to_add=[1, 10, 100, 1234]))  # Fast, yay!
print(g(numbers_to_multiply=[1, 2, 3, 4, 5]))  # Fast, yay!

1345
120
1345
120


## The caching decorators implemented in this package contain these and many other features, such as:
- Making the cache readable by other users in the system (by setting permission flags appropriately).
- Making the cache files only-readable to avoid accidental cache data deletion.
- Making the cache read-only so that collaborators cannot mess up your cache (by accidentally poluting it with unneeded data, for instance).
- Use of nested directories for the cache, to make caching faster. E.g. the cache dir would not be "_cache/f/asddnfksnfksdng" but rather "_cache/f/a/s/d/dnfksnfksdng".
- Allowing for different cache directories, so that there can be one cache per project.
- Writing out the function call signature to a file.

# To be continued!